# 神经网络的实现方法
## 搭建基本模块——神经元
在说神经网络之前，我们讨论一下神经元（Neurons），它是神经网络的基本单元。神经元先获得输入，然后执行某些数学运算后，再产生一个输出。比如一个两个输入神经元的例子：
<img src = "https://victorzhou.com/a74a19dc0599aae11df7493c718abaf9/perceptron.svg" style="zoom:60%" />

在这个神经元中，输入总共经历了3步数学运算，先将两个输入乘以权重（weight）：
$$
x1→x1 × w1 \\
x2→x2 × w2
$$

把两个结果相加，再加上一个偏置（bias）：
$$（x1 × w1）+（x2 × w2）+ b$$

最后将它们经过激活函数（activation function）处理得到输出：
$$ y = f(x1 × w1 + x2 × w2 + b) $$

激活函数的作用是将无限制的输入转换为可预测形式的输出。一种常用的激活函数是sigmoid函数：
<img src = "https://victorzhou.com/static/dd5a39500acbef371d8d791d2cd381e0/7e3cb/sigmoid.webp" style="zoom:40%" />

sigmoid函数的输出介于0和1，我们可以理解为它把$(−∞,+∞)$范围内的数压缩到$(0, 1)$以内。正值越大输出越接近1，负向数值越大输出越接近0。

举个例子，上面神经元里的权重和偏置取如下数值：$ w=[0,1]、 b = 4$ （$w=[0,1]$是$w1=0、 w2=1$的向量形式写法。）

给神经元一个输入$x=[2,3]$，可以用向量点积的形式把神经元的输出计算出来：
$$w·x+b =（x1 × w1）+（x2 × w2）+ b = 0×2+1×3+4=7 \\
y=f(w⋅X+b)=f(7)=0.999
$$

下面是以上步骤的python代码：

In [1]:
import numpy as np

def sigmoid(x):
  # Our activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    # Weight inputs, add bias, then use the activation function
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))    # 0.9990889488055994

0.9990889488055994


## 搭建神经网络
神经网络就是把一堆神经元连接在一起，下面是一个神经网络的简单举例：
<img src = "https://victorzhou.com/77ed172fdef54ca1ffcfb0bba27ba334/network.svg" style="zoom:80%" />

这个网络有2个输入、一个包含2个神经元的隐藏层（h1和h2）、包含1个神经元的输出层o1。隐藏层是夹在输入层和输出层之间的部分，一个神经网络可以有多个隐藏层。把神经元的输入向前传递获得输出的过程称为**前馈**（feedforward）。

我们假设上面的网络里所有神经元都具有相同的权重$w=[0,1]$和偏置$b=0$，激活函数都是sigmoid，那么我们会得到什么输出呢？
$$h1=h2=f(w⋅x+b)=f((0×2)+(1×3)+0)=f(3)=0.9526\\
o1=f(w⋅[h1,h2]+b)=f((0∗h1)+(1∗h2)+0)=f(0.9526)=0.7216$$

以下是实现代码：


In [2]:
import numpy as np

# ... code from previous section here

class OurNeuralNetwork:
  '''
  A neural network with:
    - 2 inputs
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)
  Each neuron has the same weights and bias:
    - w = [0, 1]
    - b = 0
  '''
  def __init__(self):
    weights = np.array([0, 1])
    bias = 0

    # The Neuron class here is from the previous section
    self.h1 = Neuron(weights, bias)
    self.h2 = Neuron(weights, bias)
    self.o1 = Neuron(weights, bias)

  def feedforward(self, x):
    out_h1 = self.h1.feedforward(x)
    out_h2 = self.h2.feedforward(x)

    # The inputs for o1 are the outputs from h1 and h2
    out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))

    return out_o1

network = OurNeuralNetwork()
x = np.array([2, 3])
print(network.feedforward(x)) # 0.7216325609518421

0.7216325609518421


## 训练神经网络
现在我们已经学会了如何搭建神经网络，现在我们来学习如何训练它，其实这就是一个优化的过程。

假设有一个数据集，包含4个人的身高、体重和性别：
<img src="https://s2.loli.net/2022/11/24/L2K9lgDJvxfz5po.png" style="zoom:80%" />

现在我们的目标是训练一个网络，根据体重和身高来推测某人的性别。
<img src="https://victorzhou.com/965173626f97e1e6b497a136d0c14ec1/network2.svg" style="zoom:80%" />

为了简便起见，我们将每个人的身高、体重减去一个固定数值，把性别男定义为0、性别女定义为1。
<img src="https://s2.loli.net/2022/11/24/GXoHO9u56cnKerA.png" style="zoom:80%" />


### 损失函数
在训练神经网络之前，我们需要有一个标准定义它到底好不好，以便我们进行改进，这就是损失（loss）。

比如用均方误差（MSE）来定义损失：$\mathrm{MSE}=\frac{1}{n} \sum_{i=1}^{n}\left(y_{t r u e}-y_{p r e d}\right)^{2}$

n是样本的数量，在上面的数据集中是4；y代表人的性别，男性是0，女性是1；$y_{true}$是变量的真实值，$y_{pred}$是变量的预测值。

顾名思义，均方误差就是所有数据方差的平均值，我们不妨就把它定义为损失函数。预测结果越好，损失就越低，训练神经网络就是将损失最小化。如果上面网络的输出一直是0，也就是预测所有人都是男性，那么损失是：
<img src="https://s2.loli.net/2022/11/24/XzbuBcwiI4NEag2.png" style="zoom:80%" />
$MSE= 1/4 * (1+0+0+1)= 0.5$

计算损失函数的代码如下：

In [6]:
import numpy as np

def mse_loss(y_true, y_pred):
  # y_true and y_pred are numpy arrays of the same length.
  return ((y_true - y_pred) ** 2).mean()

y_true = np.array([1, 0, 0, 1])
y_pred = np.array([0, 0, 0, 0])

print(mse_loss(y_true, y_pred)) # 0.5

0.5


### 反向传播
减少神经网络损失这个神经网络不够好，还要不断优化，尽量减少损失。我们知道，改变网络的权重和偏置可以影响预测值，但我们应该怎么做呢？为了简单起见，我们把数据集缩减到只包含Alice一个人的数据。于是损失函数就剩下Alice一个人的方差：
\begin{aligned}
\mathrm{MSE} &=\frac{1}{1} \sum_{i=1}^{1}\left(y_{\text {true }}-y_{\text {pred }}\right)^{2} \\
&=\left(y_{\text {true }}-y_{\text {pred }}\right)^{2} \\
&=\left(1-y_{\text {pred }}\right)^{2}
\end{aligned}
预测值是由一系列网络权重和偏置计算出来的：

<img src="https://victorzhou.com/27cf280166d7159c0465a58c68f99b39/network3.svg" style="zoom:80%" />

所以损失函数实际上是包含多个权重、偏置的多元函数：$L\left(w_{1}, w_{2}, w_{3}, w_{4}, w_{5}, w_{6}, b_{1}, b_{2}, b_{3}\right)$

如果调整一下w1，损失函数是会变大还是变小？我们需要知道偏导数$∂L/∂w1$是正是负才能回答这个问题。根据链式求导法则：
$$\frac{\partial L}{\partial w_{1}}=\frac{\partial L}{\partial y_{\text {pred }}} * \frac{\partial y_{\text {pred }}}{\partial w_{1}}$$

而$L=\left(1-y_{\text {pred }}\right)^{2}$，可以求得第一项偏导数：
$$\frac{\partial L}{\partial y_{\text {pred }}}=\frac{\partial\left(1-y_{\text {pred }}\right)^{2}}{\partial y_{\text {pred }}}=-2\left(1-y_{\text {pred }}\right)$$

接下来我们要想办法获得$y_{pred}$和w1的关系，我们已经知道神经元h1、h2和o1的数学运算规则：
$$y_{\text {pred }}=o_{1}=f\left(w_{5} h_{1}+w_{6} h_{2}+b_{3}\right)$$

实际上只有神经元h1中包含权重w1，所以我们再次运用链式求导法则：
$$
\begin{array}{c}
\frac{\partial y_{\text {pred }}}{\partial w_{1}}=\frac{\partial y_{\text {pred }}}{\partial h_{1}} * \frac{\partial h_{1}}{\partial w_{1}} \\
\frac{\partial y_{\text {pred }}}{\partial h_{1}}=w_{5} * f^{\prime}\left(w_{5} h_{1}+w_{6} h_{2}+b_{3}\right)
\end{array}
$$

然后求∂h1/∂w1
$$\begin{array}{c}
h_{1}=f\left(w_{1} x_{1}+w_{2} x_{2}+b_{1}\right) \\
\frac{\partial h_{1}}{\partial w_{1}}=x_{1} * f^{\prime}\left(w_{1} x_{1}+w_{2} x_{2}+b_{1}\right)
\end{array}$$

我们在上面的计算中遇到了2次激活函数sigmoid的导数f′(x)，sigmoid函数的导数很容易求得：
$$\begin{array}{c}
f(x)=\frac{1}{1+e^{-x}} \\
f^{\prime}(x)=\frac{e^{x}}{\left(1+e^{-x}\right)^{2}}=f(x) *(1-f(x))
\end{array}$$

总的链式求导公式：$$\frac{\partial L}{\partial w_{1}}=\frac{\partial L}{\partial y_{\text {pred }}} * \frac{\partial y_{\text {pred }}}{\partial h_{1}} * \frac{\partial h_{1}}{\partial w_{1}}$$

这种向后计算偏导数的系统称为反向传播（backpropagation）。上面的数学符号太多，下面我们带入实际数值来计算一下h1、h2和o1。
### 偏导数计算示例
<img src = "https://s2.loli.net/2022/11/24/C7MslnpRaBg4NFm.png" style="zoom:70%" />

把所有权重初始化为1以及所有偏置初始化为0。如果我们通过网络进行前馈传递，我们会得到：
$$
h1=f(x1⋅w1+x2⋅w2+b1)=0.0474 \\
h2=f(w3⋅x3+w4⋅x4+b2)=0.0474 \\
o1=f(w5⋅h1+w6⋅h2+b3)=f(0.0474+0.0474+0)=f(0.0948)=0.524 \\
$$
神经网络的输出y=0.524，没有显示出强烈的是男（1）是女（0）的证据。现在的预测效果还很不好。

我们再计算一下当前网络的偏导数∂L/∂w1：
$$
\begin{aligned}
\frac{\partial L}{\partial w_{1}} &=\frac{\partial L}{\partial y_{\text {pred }}} * \frac{\partial y_{\text {pred }}}{\partial h_{1}} * \frac{\partial h_{1}}{\partial w_{1}} \\
\frac{\partial L}{\partial y_{\text {pred }}}&=-2\left(1-y_{\text {pred }}\right) \\
&=-2(1-0.524) \\
&=-0.952 \\
\frac{\partial y_{\text {pred }}}{\partial h_{1}} &=w_{5} * f^{\prime}\left(w_{5} h_{1}+w_{6} h_{2}+b_{3}\right) \\
&=1 * f^{\prime}(0.0474+0.0474+0) \\
&=f(0.0948) *(1-f(0.0948)) \\
&=0.249 \\
\frac{\partial h_{1}}{\partial w_{1}} &=x_{1} * f^{\prime}\left(w_{1} x_{1}+w_{2} x_{2}+b_{1}\right) \\
&=-2 * f^{\prime}(-2+-1+0) \\
&=-2 * f(-3) *(1-f(-3)) \\
&=-0.0904 \\
\frac{\partial L}{\partial w_{1}} &=-0.952 * 0.249 *-0.0904 \\
&=0.0214
\end{aligned}
$$

这个结果告诉我们：如果增大w1，损失函数L会有一个非常小的增长。

### 随机梯度下降
随机梯度下降下面将使用一种称为随机梯度下降（SGD）的优化算法，来训练网络。

经过前面的运算，我们已经有了训练神经网络所有数据。但是该如何操作？SGD定义了改变权重和偏置的方法：$w_{1} \leftarrow w_{1}-\eta \frac{\partial L}{\partial w_{1}}$ 

其中η是一个常数，称为学习率（learning rate），它决定了我们训练网络速率的快慢。将w1减去η·∂L/∂w1，就等到了新的权重w1。

当∂L/∂w1是正数时，w1会变小；当∂L/∂w1是负数 时，w1会变大。

如果我们用这种方法去逐步改变网络的权重w和偏置b，损失函数会缓慢地降低，从而改进我们的神经网络。训练流程如下：
- 从数据集中选择一个样本；
- 计算损失函数对所有权重和偏置的偏导数；
- 使用更新公式更新每个权重和偏置；
- 回到步骤1

### Python实现神经网络

In [4]:
import numpy as np

def sigmoid(x):
  # Sigmoid activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
  # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)

def mse_loss(y_true, y_pred):
  # y_true and y_pred are numpy arrays of the same length.
  return ((y_true - y_pred) ** 2).mean()

class OurNeuralNetwork:
  '''
  A neural network with:
    - 2 inputs
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)

  *** DISCLAIMER ***:
  The code below is intended to be simple and educational, NOT optimal.
  Real neural net code looks nothing like this. DO NOT use this code.
  Instead, read/run it to understand how this specific network works.
  '''
  def __init__(self):
    # Weights
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Biases
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
    return o1

  def train(self, data, all_y_trues):
    '''
    - data is a (n x 2) numpy array, n = # of samples in the dataset.
    - all_y_trues is a numpy array with n elements.
      Elements in all_y_trues correspond to those in data.
    '''
    learn_rate = 0.1
    epochs = 1000 # number of times to loop through the entire dataset

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # --- Do a feedforward (we'll need these values later)
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        # --- Calculate partial derivatives.
        # --- Naming: d_L_d_w1 represents "partial L / partial w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Neuron o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Neuron h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Neuron h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # --- Update weights and biases
        # Neuron h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Neuron h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Neuron o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # --- Calculate total loss at the end of each epoch
      if epoch % 10 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        print("Epoch %d loss: %.3f" % (epoch, loss))

# Define dataset
data = np.array([
  [-2, -1],  # Alice
  [25, 6],   # Bob
  [17, 4],   # Charlie
  [-15, -6], # Diana
])
all_y_trues = np.array([
  1, # Alice
  0, # Bob
  0, # Charlie
  1, # Diana
])

# Train our neural network!
network = OurNeuralNetwork()
network.train(data, all_y_trues)

Epoch 0 loss: 0.277
Epoch 10 loss: 0.159
Epoch 20 loss: 0.099
Epoch 30 loss: 0.070
Epoch 40 loss: 0.053
Epoch 50 loss: 0.042
Epoch 60 loss: 0.035
Epoch 70 loss: 0.029
Epoch 80 loss: 0.025
Epoch 90 loss: 0.022
Epoch 100 loss: 0.020
Epoch 110 loss: 0.018
Epoch 120 loss: 0.016
Epoch 130 loss: 0.015
Epoch 140 loss: 0.013
Epoch 150 loss: 0.012
Epoch 160 loss: 0.012
Epoch 170 loss: 0.011
Epoch 180 loss: 0.010
Epoch 190 loss: 0.009
Epoch 200 loss: 0.009
Epoch 210 loss: 0.008
Epoch 220 loss: 0.008
Epoch 230 loss: 0.008
Epoch 240 loss: 0.007
Epoch 250 loss: 0.007
Epoch 260 loss: 0.007
Epoch 270 loss: 0.006
Epoch 280 loss: 0.006
Epoch 290 loss: 0.006
Epoch 300 loss: 0.006
Epoch 310 loss: 0.005
Epoch 320 loss: 0.005
Epoch 330 loss: 0.005
Epoch 340 loss: 0.005
Epoch 350 loss: 0.005
Epoch 360 loss: 0.005
Epoch 370 loss: 0.004
Epoch 380 loss: 0.004
Epoch 390 loss: 0.004
Epoch 400 loss: 0.004
Epoch 410 loss: 0.004
Epoch 420 loss: 0.004
Epoch 430 loss: 0.004
Epoch 440 loss: 0.004
Epoch 450 loss: 0.004

随着学习过程的进行，损失函数逐渐减小。
<img src = "https://victorzhou.com/static/99e7886af56d6f41b484d17a52f9241b/111e4/loss.webp" style="zoom:80%" />

现在我们可以用它来推测出每个人的性别：

In [5]:
# Make some predictions
emily = np.array([-7, -3]) # 128 pounds, 63 inches
frank = np.array([20, 2])  # 155 pounds, 68 inches
print("Emily: %.3f" % network.feedforward(emily)) # 0.951 - F
print("Frank: %.3f" % network.feedforward(frank)) # 0.039 - M


Emily: 0.964
Frank: 0.039


### 我们学习到了

- 引入神经元，神经网络的构建块。
- 在我们的神经元中使用了sigmoid激活功能。
- 看到神经网络只是连接在一起的神经元。
- 创建了一个数据集，其中体重和身高作为输入（或要素），性别作为输出（或标注）。
- 了解了损失函数和均方误差 （MSE） 损失。
- 意识到训练网络只是最大限度地减少其损失。
- 使用反向传播来计算偏导数。
- 使用随机梯度下降（SGD）来训练我们的网络。

### 日后的学习

- 用更大更好的机器学习库搭建神经网络，如Tensorflow、Keras、PyTorch
- 在浏览器中的直观理解神经网络（https://playground.tensorflow.org/ ）
- 学习sigmoid以外的其他激活函数，如ReLU、Leaky ReLU、tanh、ELU
- 学习SGD以外的其他优化器，如Adam、RMSProp、AdaGrad
- 学习卷积神经网络（CNN），CNN彻底改变了计算机视觉领域，并且可以非常强大。
- 学习递归神经网络（RNN），它通常用于自然语言处理（NLP）。
